# <font color='blue'>Classificação do Modelo</font>



## <font color = 'orange'>O que é classificação</font>
---

A classificação é uma técnica de previsão de alguma variável qualitativa que será prevista por outra 

## <font color='orange'>Critérios utilizados</font>
---

Como explicado no Jupyter da [`regressão linear`](Books.ipynb), vamos mudar um pouco o modelo e tentar realizar uma classificação do 'book_rating' para entender como o modelo responde e se nossa previsão pode ficar melhor dessa maneira.

Os critérios utilizados foram:
- 0 até 2,5 - Muito ruim ou não avaliado
- 2,5 até 3,5 - Ruim
- 3,5 até 4 - Médio
- 4 até 4,35 - Bom
- 4,35 até 5 - Muito bom

Usamos essas faixas para evitar que houvesse uma faixa com concentração quase total dos livros, tentando contornar esse problema da melhor maneira.

In [ ]:
#Importando bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import seaborn
import statsmodels.api as sm

In [ ]:
#Lendo CSV
csv = pd.read_csv('book_data.csv')

In [ ]:
# Retirando todas as variáveis que não serão utilizadas
csv.drop(['book_isbn','book_desc','book_edition','book_rating_count','image_url'],axis=1,inplace=True)
# Retirando os livros com informações faltando
csv=csv.dropna()
csv.head()

In [ ]:
# Fazendo a classificação dos livros
lista = ['muito ruim ou não avaliado', 'ruim', 'médio', 'bom', 'muito bom']
avaliacoes = []

for i in csv['book_rating'].values:
    if i>=0 and i<2.5:
        avaliacoes.append(lista[0])
    elif i>=2.5 and i<3.5:
        avaliacoes.append(lista[1])
    elif i>=3.5 and i<4:
        avaliacoes.append(lista[2])
    elif i>=4  and i<4.35:
        avaliacoes.append(lista[3])
    elif i>=4.35 and i<=5:
        avaliacoes.append(lista[4])

In [ ]:
# Entendendo como as classificações estão distribuidas
j = pd.Series(avaliacoes)
j = j.value_counts()
j

In [ ]:
# Juntando as classificações no dataframe
avaliacoes_ = pd.DataFrame({'avaliações':avaliacoes}, index = csv.index)
csv = csv.join(avaliacoes_,how='inner')
csv.drop(['book_rating'],axis=1, inplace=True)

In [ ]:
csv.head()

In [ ]:
# Reajustando as variáveis que serão utilizadas
def limpando(csv):
    # Ajeitando as variáveis
    ar = csv['genres'].tolist()
    ar1 = []
    iniciais = []
    paginas = []

    # Escolhendo como gênero literário apenas o primeiro listado.
    for i in ar:
        if i == "":
            ar1.append("")
        else:
            a = i.split('|')
            ar1.append(a[0])

    # Selecionando apenas a inicial dos nomes dos autores
    for i in csv['book_authors']:
        iniciais.append(i[0])

    # Apenas o número de páginas aparecerá, sem a palavra "pages"
    for i in csv['book_pages']:
        numero = i.split(' ')
        paginas.append(int(numero[0]))
    return ar1,paginas,iniciais
ar1,paginas,iniciais = limpando(csv)
print(ar1[0:10],paginas[0:10],iniciais[0:10])

In [ ]:
# Criando colunas no DataFrame para as iniciais dos autores, os números de páginas e o principal gênero do livro
ar1=pd.DataFrame({'genero':ar1},index = csv.index)
iniciais_ = pd.DataFrame({'iniciais':iniciais}, index = csv.index)
paginas_ = pd.DataFrame({'paginas':paginas}, index = csv.index)

In [ ]:
# Juntando as iniciais dos autores, número de páginas e o principal gênero
csv = csv.join(ar1,how='inner')
csv = csv.join(iniciais_,how='inner')
csv = csv.join(paginas_,how='inner')

In [ ]:
# Retirando as colunas que foram ajustadas
csv.drop(['book_authors','book_pages','book_title','genres'],axis=1,inplace=True)

In [ ]:
csv.head(10)

In [ ]:
# Separando entre o resultado esperado, variáveis qualitativas e variáveis quantitativas
y=csv.avaliações
num = csv[['paginas','book_review_count']]
qual = csv[['book_format', 'genero', 'iniciais']]

In [ ]:
# Transformando qualitativas em quantitativas por meio do OneHotEncoder
encoder = OneHotEncoder()
capa_encoded = encoder.fit_transform(qual)
x_num = num.values
x_encoded = capa_encoded.toarray()
X = np.concatenate((x_num,x_encoded), axis=1)

In [ ]:
# Realizando o modelo
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y)

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(accuracy_score(y_test, y_pred))

## <font color='orange'>Conclusão</font>
---

Portanto, no nosso modelo, dada uma das classificações que fizemos, temos 53,8% de acertar ele dada nossas variáveis. É uma probabilidade razoável dado que temos 5 classificações e um 'chute' teria somente 20% de chance de acerto.